## Basic setup and directory structure

In [ ]:
import os

BASE_DIR = '/home/rithomas'
# folder containing MIDI and/or MusicXML files. can have child folders.
RAW_DATA = os.path.join(BASE_DIR, 'data', 'clean_midi')
# TFRecord file that will contain NoteSequence protocol buffers.
SEQUENCES_TFRECORD = os.path.join(BASE_DIR, 'cache', 'performanceRNN', 'data', 'notesequences.tfrecord')
# Model config must be one of ['basic_improv', 'attention_improv', 'chord_pitches_improv']
PERFORMANCE_RNN_CONFIG = 'chord_pitches_improv'
# Model directory for storing weights and tensorboard stuff
PERFORMANCE_RNN_DIR = os.path.join(BASE_DIR, 'cache', 'performanceRNN', 'model')
# Dataset of Sequence Examples for the ImprovRNN Model
PERFORMANCE_RNN_DATASET = os.path.join(BASE_DIR, 'cache', 'performanceRNN')
# Training sequence examples
TRAINING_EXAMPLES = os.path.join(PERFORMANCE_RNN_DATASET, 'training_performances.tfrecord')
# Dir for testing outputs
OUTPUTS_PATH = os.path.join(PERFORMANCE_RNN_DATASET, 'outputs')
# Final bundled model
BUNDLE_DIR = os.path.join(PERFORMANCE_RNN_DIR, PERFORMANCE_RNN_CONFIG + '.mag')

if not os.path.exists(SEQUENCES_TFRECORD):
    !python3 "magenta/scripts/convert_dir_to_note_sequences.py" \
      --input_dir=$RAW_DATA \
      --output_file=$SEQUENCES_TFRECORD \
      --recursive

## Create PerformanceRNN Dataset

In [ ]:
# CONFIG=<one of 'performance', 'performance_with_dynamics', etc.>
!python3 "magenta/models/performance_rnn/performance_rnn_create_dataset.py" \
    --config=$PERFORMANCE_RNN_CONFIG \
    --input=$SEQUENCES_TFRECORD \
    --output_dir=$PERFORMANCE_RNN_DATASET \
    --eval_ratio=0.10

## Train PerformanceRNN on Dataset 

In [ ]:
!python3 "magenta/models/performance_rnn/performance_rnn_train.py" \
    --config=$PERFORMANCE_RNN_CONFIG \
    --run_dir=$PERFORMANCE_RNN_DIR \
    --sequence_example_file=$TRAINING_EXAMPLES

## Generate some outputs for testing

In [ ]:
!python3 "magenta/models/performance_rnn/performance_rnn_generate.py" \
    --config=$PERFORMANCE_RNN_CONFIG \
    --run_dir=$PERFORMANCE_RNN_DIR \
    --output_dir=$OUTPUTS_PATH \
    --num_outputs=10 \
    --num_steps=3000 \
    --primer_melody="[60,62,64,65,67,69,71,72]"

## Save model if you like it!

In [ ]:
!python3 "magenta/modes/performance_rnn/performance_rnn_generate.py" \
    --config=$PERFORMANCE_RNN_CONFIG \
    --run_dir=$PERFORMANCE_RNN_DIR \
    --bundle_file=$BUNDLE_DIR \
    --save_generator_bundle